In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_excel("Online Retail.xlsx")
df.head()
df1 = df

In [4]:
df1.Country.nunique()

38

In [5]:
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [6]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [7]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

In [8]:
df1 = df1[pd.notnull(df1['CustomerID'])]

In [9]:
df1.Quantity.min()

-80995

In [10]:
df1 = df1[(df1['Quantity']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [11]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  16649
StockCode :  3645
Description :  3844
Quantity :  294
InvoiceDate :  15615
UnitPrice :  403
CustomerID :  3921
Country :  1


In [12]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']

In [13]:
df1['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [14]:
df1['InvoiceDate'].max()

Timestamp('2011-12-09 12:49:00')

In [15]:
import datetime as dt
NOW = dt.datetime(2011,12,10)
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [16]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, 'InvoiceNo': lambda x: len(x), 'TotalPrice': lambda x: x.sum()})
rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

In [17]:
rfmTable.head()

,monetary_value,frequency,recency
CustomerID,,,
12346.0,77183.60,1,325
12747.0,4196.01,103,2
12748.0,33719.73,4596,0
12749.0,4090.88,199,3
12820.0,942.34,59,3


In [19]:
first_customer = df1[df1['CustomerID'] == 12346.0]
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


In [20]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [21]:
segmented_rfm = rfmTable

In [22]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [23]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,monetary_value,frequency,recency,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,77183.60,1,325,4,4,1
12747.0,4196.01,103,2,1,1,1
12748.0,33719.73,4596,0,1,1,1
12749.0,4090.88,199,3,1,1,1
12820.0,942.34,59,3,1,2,2


In [25]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,monetary_value,frequency,recency,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,77183.60,1,325,4,4,1,441
12747.0,4196.01,103,2,1,1,1,111
12748.0,33719.73,4596,0,1,1,1,111
12749.0,4090.88,199,3,1,1,1,111
12820.0,942.34,59,3,1,2,2,122


In [26]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,monetary_value,frequency,recency,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,259657.30,431,0,1,1,1,111
17450.0,194550.79,337,8,1,1,1,111
17511.0,91062.38,963,2,1,1,1,111
16684.0,66653.56,277,4,1,1,1,111
14096.0,65164.79,5111,4,1,1,1,111
13694.0,65039.62,568,3,1,1,1,111
15311.0,60767.90,2379,0,1,1,1,111
13089.0,58825.83,1818,2,1,1,1,111
15769.0,56252.72,130,7,1,1,1,111
